In [3]:
import pandas as pd
import re
import dash
from dash import dash_table, dcc, html
from dash.dependencies import Input, Output

# Read the log file
with open("example2.log", "r") as file:
    log_lines = file.readlines()

# Prepare lists to store the extracted information
actions = []
prompt_tokens = []
completion_tokens = []
costs = []

# Parse the log file
for line in log_lines:
    if "INFO:__main__:" in line:
        action = line.split(":")[2]
        actions.append(action)

    if "tokens" in line:
        prompt_token = int(re.search(r'Prompt tokens: (\d+)', line).group(1))
        completion_token = int(re.search(r'Completion tokens: (\d+)', line).group(1))
        cost = float(re.search(r'Cost: (\d+.\d+)', line).group(1))
        
        prompt_tokens.append(prompt_token)
        completion_tokens.append(completion_token)
        costs.append(cost)

# Make sure all lists are of same length
min_len = min(len(actions), len(prompt_tokens), len(completion_tokens), len(costs))

actions = actions[:min_len]
prompt_tokens = prompt_tokens[:min_len]
completion_tokens = completion_tokens[:min_len]
costs = costs[:min_len]

# Create a DataFrame
df = pd.DataFrame({"Action": actions, "Prompt Tokens": prompt_tokens, 
                   "Completion Tokens": completion_tokens, "Cost": costs})

# Create a Dash app
app = dash.Dash(__name__)

# Define the app layout
app.layout = html.Div([
    dash_table.DataTable(
        id='table',
        columns=[{"name": i, "id": i} for i in df.columns],
        data=df.to_dict('records'),
    )
])

# Run the app
if __name__ == '__main__':
    app.run_server(debug=True)


In [5]:
import pandas as pd
import re
import dash
from dash import dash_table, dcc, html

# Read the log file
with open("example2.log", "r") as file:
    log_lines = file.readlines()

# Prepare lists to store the extracted information
actions = []
prompt_tokens = []
completion_tokens = []
costs = []

# Parse the log file
for line in log_lines:
    if "INFO:__main__:" in line:
        action = line.split(":")[2].strip()
        actions.append(action)

    if "tokens" in line:
        prompt_token = int(re.search(r'Prompt tokens: (\d+)', line).group(1))
        completion_token = int(re.search(r'Completion tokens: (\d+)', line).group(1))
        cost = float(re.search(r'Cost: (\d+.\d+)', line).group(1))

        prompt_tokens.append(prompt_token)
        completion_tokens.append(completion_token)
        costs.append(cost)

# Make sure all lists are of same length
min_len = min(len(actions), len(prompt_tokens), len(completion_tokens), len(costs))

actions = actions[:min_len]
prompt_tokens = prompt_tokens[:min_len]
completion_tokens = completion_tokens[:min_len]
costs = costs[:min_len]

# Create a DataFrame
df = pd.DataFrame({"Action": actions, "Prompt Tokens": prompt_tokens, 
                   "Completion Tokens": completion_tokens, "Cost": costs})

# Group by 'Action', sum 'Cost', 'Prompt Tokens', and 'Completion Tokens'
df_grouped = df.groupby('Action').sum().reset_index()

# Create a Dash app
app = dash.Dash(__name__)

# Define the app layout
app.layout = html.Div([
    dash_table.DataTable(
        id='table',
        columns=[{"name": i, "id": i} for i in df_grouped.columns],
        data=df_grouped.to_dict('records'),
    )
])

# Run the app
if __name__ == '__main__':
    app.run_server(debug=True)
